In [2]:
# Obligatory assignment 1
# Students: skn003, tre081

import numpy as np
import pandas as pd
import re
import regex
import nltk
import matplotlib.pyplot as plt
#nltk.download('stopwords')
from nltk.corpus import stopwords

# Storing the training dataset for further use
data = pd.read_csv('train.csv', delimiter = ',')

pd.options.display.max_colwidth = 100

display(data.head(4))

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 \nThe tension between intelligence analysts a...",1
3,3,15 Civilians Killed In Single US Airstrike Have Been Identified,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstrike Have Been Identified The rate at which civilia...,1


In [3]:
# Printing the dimensions of the dataset as a reference point for upcoming trimming
print(data.shape)

(20800, 5)


In [4]:
# Removing empty rows
filter = data['text'].str.contains('', regex = True, na = False)

data = data[filter]
print(data.shape)

(20761, 5)


In [5]:
# Creating a new column with the combined data in the title, author and text columns
data['total']=data['title']+' '+data['author']+data['text']

In [6]:
data=data.fillna('')

In [7]:
filter = []

# Removing all rows with non-latin characters
for index, row in data.iterrows():
    reg = regex.match(r'[\p{Latin}\p{posix_punct}]+', row['total'])
    if reg == None:
        filter.append(False)
    else:
        filter.append(True)

In [8]:
filter = np.array(filter)

filter

array([ True,  True,  True, ...,  True,  True,  True])

In [9]:
# Applying filter to the dataset
data = data[filter]

# Printing results of the filter
print("after non-latin removal:", data.shape)

after non-latin removal: (17835, 6)


In [10]:
# Removing newline characters
data['text'].replace(regex=True, inplace=True,to_replace='\n', value='')

# Exporting the dataset to csv format for storage
data.to_csv('newtrain.csv')

In [11]:
# Creating a list with only the texts and none of the metadata
texts = data['text'].tolist()

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

# Building a vocabulary from all the texts
vectorizer.fit(texts)
vocabulary = vectorizer.vocabulary_ # Storing vocabulary in new variable for easier use
print("Before stopwords removal:", len(vocabulary)) # Printing the size of the vocabulary prior to removal of stopwords

Before stopwords removal: 147069


In [13]:
#import nltk
#nltk.download('stopwords')

# Retrieving a list of English stopwords
stopWords = set(stopwords.words('english'))

# Creating a copy of vocabulary for iteration
temp_dict = dict(vocabulary)
for key in temp_dict:
    if key in stopWords:
        del vocabulary[key] # Removing English stopwords from actual vocabulary

In [14]:
# Printing results of the stopword removal
print("After stopwords removal:", len(vocabulary))

After stopwords removal: 146924


In [15]:
# Exporting the vocabulary to a csv file
pd.DataFrame([vocabulary]).to_csv('Vocabulary.csv', sep=',')

In [16]:
# Assignment 2 prep

temp_dict = dict(vocabulary).keys()
for key in temp_dict:
    if not re.match(r'[^\W\d]*$', key):
        del vocabulary[key]


print(len(vocabulary))

141184


In [17]:
pd.DataFrame([vocabulary]).to_csv('Vocabulary2.csv', sep=',')

In [18]:
temp_dict = dict(vocabulary).keys()
for key in temp_dict:
    for c in ['_', ')', '(', '{', '}', '[', ']', '/', ',', '.']:
        if c in key:
            del vocabulary[key]


print(len(vocabulary))

140810


In [18]:
temp_dict = dict(vocabulary).keys()
new_vocab = {}
vocab_reliable = {}
vocab_unreliable = {}
for word in temp_dict:
    count = 0
    unreliable = 0
    reliable = 0
    for text in data['total']:
        count += text.count(word)
        
    
    if count > 1000:
        new_vocab[word] = count
        # del vocabulary[key]

In [20]:
print(len(new_vocab))

15974


In [21]:
df = pd.DataFrame(new_vocab, columns = ['Word', 'Count'])
pd.DataFrame([vocabulary]).to_csv('Vocabulary_small.csv', sep=',')

In [20]:
train = pd.read_csv('newtrain.csv', delimiter = ',')

In [21]:
train.head()

,Unnamed: 0,id,title,author,text,label,total
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It Darrell LucusH...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart Daniel J. FlynnEver get the feeling your..."
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 The tension between intelligence analysts and...",1,Why the Truth Might Get You Fired Consortiumnews.comWhy the Truth Might Get You Fired October 29...
3,4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print An Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Guar...,1,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery Ho...
4,5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for B...",0,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...


In [22]:
train = train.drop(columns=['total'])

In [23]:
train.head()

,Unnamed: 0,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Luc...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart",Daniel J. Flynn,Ever get the feeling your life circles the roundabout rather than heads in a straight line towar...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, 2016 The tension between intelligence analysts and...",1
3,4,4,Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery,Howard Portnoy,Print An Iranian woman has been sentenced to six years in prison after Iran’s Revolutionary Guar...,1
4,5,5,Jackie Mason: Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms (...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voice of Reason. [In this week’s exclusive clip for B...",0


In [24]:
data.to_csv('newtrain.csv')